In [5]:
import numpy as np
import numpy.random as rand

In [30]:
#create environment
#take input from user
while True:
  try:
    dimension = int(input("Enter dimension of board: "))
    break
  except ValueError:
    print("Must be an integer")

while True: 
  try:
    num_mines = int(input("Enter number of mines: "))
    if num_mines > dimension**2:
      raise ValueError
    break
  except ValueError:
    print("Must be an integer and less than dimension^2")



Enter dimension of board: 10
Enter number of mines: 25


In [31]:
#initialize board
board = np.zeros((dimension, dimension))

#initialize mines
counter = 0
mines_set = set([])
while counter < num_mines:
  x = rand.randint(0, dimension)
  y = rand.randint(0, dimension)
  if (x,y) in mines_set:
    continue
  else:
    mines_set.add((x,y))
    board[x][y] = 1
    counter += 1

print(board)

[[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 1. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [1. 1. 0. 1. 0. 1. 0. 0. 1. 1.]
 [1. 0. 1. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 1. 0. 0. 1. 0. 1. 0.]
 [1. 0. 0. 0. 1. 0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 1. 0. 0. 0. 0. 0.]]


In [8]:
#clue that our agent will receive
#returns the total sum of the neighbors of current_position
def clue(current_position, board,  dimension):
  x = current_position[0]
  y = current_position[1]
  sum = 0

  if x+1 < dimension:
    sum += board[x+1][y]
    if y+1 < dimension:
      sum += board[x+1][y+1]
    if y-1 >= 0:
      sum += board[x+1][y-1]

  if x-1 >= 0:
    sum += board[x-1][y]
    if y+1 < dimension:
      sum += board[x-1][y+1]
    if y-1 >= 0:
      sum += board[x-1][y-1]

  if y+1 < dimension:
    sum += board[x][y+1]
  
  if y-1 >= 0:
    sum += board[x][y-1]

  return sum
  

In [9]:
def neighbors(current_position, board,  dimension):
  x = current_position[0]
  y = current_position[1]
  nodes = set([])

  if x+1 < dimension:
    nodes.add((x+1,y))
    if y+1 < dimension:
      nodes.add((x+1,y+1))
    if y-1 >= 0:
      nodes.add((x+1,y-1))

  if x-1 >= 0:
    nodes.add((x-1,y))
    if y+1 < dimension:
      nodes.add((x-1,y+1))
    if y-1 >= 0:
      nodes.add((x-1,y-1))

  if y+1 < dimension:
    nodes.add((x,y+1))
  
  if y-1 >= 0:
    nodes.add((x,y-1))

  return nodes

In [20]:
def safe_squares(current_position, board,  dimension):
  x = current_position[0]
  y = current_position[1]
  total = 0

  if x+1 < dimension:
    total += 1
    if y+1 < dimension:
      total += 1
    if y-1 >= 0:
      total += 1
  if x-1 >= 0:
    total += 1
    if y+1 < dimension:
      total += 1
    if y-1 >= 0:
      total += 1
  if y+1 < dimension:
    total += 1
  if y-1 >= 0:
    total += 1

  mines = clue(current_position, board,  dimension)
  safe = total - mines

  return safe


In [27]:
def basic_algorithm(board, dimension):
  mines = set([])
  explored_set = set([])
  neighbor_fringe = set([])
  unexplored_set = set([])
  death = 0
  for a in range(dimension):
    for b in range(dimension):
      unexplored_set.add((a,b))
  #(coords,mine/safe,clue#)
  #implement while loop here for each pair in board that isn't in explored space
  # -- not sure about where to put the random point check? maybe at the end after initializing it? --
  while len(unexplored_set) > 0:
    coord = unexplored_set.pop()
    neighbor_fringe.add(coord)
    if board[coord[0]][coord[1]] == 1:
      death += 1
      explored_set.add(coord)
      mines.add(coord)
      #break
    if board[coord[0]][coord[1]] == 0:
      #explored_set.add(coord)
      clue_num = clue(coord, board, dimension)
      #safe = safe_squares((x,y), board, dimension)
      neighbor_set = neighbors(coord, board, dimension)
      mine_count = 0
      hidden = 0
      hidden_set = set([])
      safe = 0
      safe_set = set([])
      for i in neighbor_set:
        if i in mines:
          mine_count += 1
        if i in unexplored_set:
          hidden += 1
          hidden_set.add(i)

      for j in neighbor_set:
        if j not in unexplored_set and j not in mines:
          safe += 1
          safe_set.add(i)

      #first condition
      if clue_num - mine_count == hidden:
        for j in hidden_set:
          mines.add(j)
          unexplored_set.remove(j)

    #second condition
      if len(neighbor_set) - clue_num - safe == hidden:
        for k in hidden_set:
          neighbor_fringe.add(k)
          unexplored_set.remove(k) 
  print(death)
  return mines   


In [32]:
mines = basic_algorithm(board,dimension)
board_new = np.zeros((dimension, dimension))
#initialize mines
counter = 0
for i in mines:
  board_new[i[0]][i[1]] = 1

print(board_new)

21
[[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 1. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [1. 1. 0. 1. 0. 1. 0. 0. 1. 1.]
 [1. 0. 1. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 1. 0. 0. 1. 0. 1. 0.]
 [1. 0. 0. 0. 1. 0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 1. 0. 0. 0. 0. 0.]]


In [33]:
print(board)

[[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 1. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [1. 1. 0. 1. 0. 1. 0. 0. 1. 1.]
 [1. 0. 1. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 1. 0. 0. 1. 0. 1. 0.]
 [1. 0. 0. 0. 1. 0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 1. 0. 0. 0. 0. 0.]]
